In [ ]:
import os
import glob
import SimpleITK as sitk
import numpy as np
from einops import rearrange

In [ ]:
# 1. Obtener los IDs de caso y los slices disponibles a partir de los archivos en la carpeta CTP
ctp_folder = "/data/dev/perfu-net-1/data/train/CTP"
ctp_files = glob.glob(os.path.join(ctp_folder, "case_*.npy"))
ctp_slices = {}  # Diccionario: key = case_id, value = set de slices (ej: "01", "02", ...)

for file in ctp_files:
    base = os.path.basename(file)  # ejemplo: "case_12_04.npy" o "case_1_04.npy"
    parts = base.split("_")
    if len(parts) >= 3:
        # Convertir el ID a dos dígitos (por ejemplo, "1" -> "01")
        case_id = parts[1].zfill(2)
        # Extraer el número de slice removiendo la extensión .npy (ej: "04.npy" -> "04")
        slice_num = os.path.splitext(parts[2])[0]
        if case_id not in ctp_slices:
            ctp_slices[case_id] = set()
        ctp_slices[case_id].add(slice_num)

print("Casos presentes en CTP:", list(ctp_slices.keys()))

# 2. Buscar todas las máscaras en TRAINING que tengan "OT" en su path
training_path = "/data/ISLES-2018/TRAINING"
mask_files = glob.glob(os.path.join(training_path, "case_*", "**", "*OT*.nii"), recursive=True)

# Carpeta de salida para las máscaras
mask_output_folder = "/data/dev/perfu-net/data/train/MASK"
os.makedirs(mask_output_folder, exist_ok=True)

# 3. Procesar cada archivo de máscara encontrado
for mask_file in mask_files:
    # Extraer el ID de caso a partir de la ruta. Se asume que el path contiene una carpeta con formato "case_X" o "case_XX"
    parts = mask_file.split(os.sep)
    case_name = None
    for part in parts:
        if part.startswith("case_"):
            case_name = part  # Ej: "case_12" o "case_1"
            break

    if case_name is None:
        print(f"No se encontró el ID de caso en {mask_file}. Se omite.")
        continue

    # Extraer el número de caso y convertirlo a dos dígitos
    patient_id_raw = case_name.split("_")[1]
    patient_id = patient_id_raw.zfill(2)
    
    # Solo procesar si el caso también está en CTP (con slices disponibles)
    if patient_id not in ctp_slices:
        print(f"Paciente {patient_id} no está en CTP. Se omite {mask_file}.")
        continue

    print(f"Procesando máscara para paciente {patient_id}: {mask_file}")
    
    # 3.1. Cargar la máscara usando SimpleITK y convertir a array NumPy
    mask_image = sitk.ReadImage(mask_file)
    mask_array = sitk.GetArrayFromImage(mask_image)
    # Por defecto, mask_array tiene forma (d, h, w) donde d es la cantidad de slices.
    # Reorganizamos a (h, w, d) para que z=0 corresponda a case_XX_01
    mask_array = rearrange(mask_array, 'd h w -> h w d')
    
    # Obtener el conjunto de slices disponibles para este paciente en CTP
    available_slices = ctp_slices[patient_id]
    
    # 3.2. Dividir la máscara por cada slice (componente z)
    num_slices = mask_array.shape[2]
    for z in range(num_slices):
        # El número de slice se formatea como z+1 con 2 dígitos (z=0 -> "01")
        slice_num = str(z+1).zfill(2)
        if slice_num not in available_slices:
            print(f"Slice {slice_num} no está en CTP para paciente {patient_id}. Se omite.")
            continue
        slice_mask = mask_array[:, :, z]
        # Nombrado: "case_{patient_id}_{slice_num}.npy"
        out_filename = f"case_{patient_id}_{slice_num}.npy"
        out_path = os.path.join(mask_output_folder, out_filename)
        np.save(out_path, slice_mask)
        print(f"Guardado slice z={z} (archivo {out_filename}) en {out_path}")
